In [364]:
import ee
service_account = 'clgx-gis-app-dev-06e3@appspot.gserviceaccount.com'
credentials = ee.ServiceAccountCredentials(service_account, 'clgx-gis-app-dev-06e3-a1a85a16331a.json')
ee.Initialize(credentials)

In [365]:
import geopandas as gpd
import geojson
from shapely import wkt
from datetime import datetime

In [366]:

startDate = '2020-06-01'
endDate = '2020-06-16'

In [367]:
bbox = gpd.read_file('boulder_bounds.shp')

In [368]:
bbox

,NoteType,Name,Notes,created_us,created_da,last_edite,last_edi_1,Shape_Leng,Shape_Area,geometry
0,0,None,None,ABREUNIG,2023-10-17,ABREUNIG,2023-10-17,0.207454,0.00243,"POLYGON Z ((-105.23135 40.05088 0.00000, -105...."


In [369]:
coords = list(bbox['geometry'][0].exterior.coords)

# drop z values
coords = [list(c[:2]) for c in coords]
coords

[[-105.23134908199995, 40.05088386700004],
 [-105.16393977399997, 40.05000842100003],
 [-105.16327276799996, 40.014573720000044],
 [-105.23180764799997, 40.01482384700006],
 [-105.23134908199995, 40.05088386700004]]

In [370]:
aoi = ee.Geometry.Polygon(coords)

In [371]:
s1 = ee.ImageCollection('COPERNICUS/S1_GRD')
images = (s1.filterDate(startDate, endDate).
            filterMetadata('instrumentMode', 'equals', 'IW').
            filterMetadata('orbitProperties_pass', 'equals', 'DESCENDING').
            filterBounds(aoi).
            filter(ee.Filter.eq('resolution_meters', 10))
)
lst = images.toList(images.size())    
i1 = ee.Image(lst.get(0))
i2 = ee.Image(lst.get(1))
i3 = ee.Image(lst.get(2))

i1_props = i1.getInfo()['properties']
i2_props = i2.getInfo()['properties']
i3_props = i3.getInfo()['properties']


In [372]:
images_meta_list

[{'type': 'Image',
  'bands': [{'id': 'VV',
    'data_type': {'type': 'PixelType', 'precision': 'double'},
    'dimensions': [29008, 21488],
    'crs': 'EPSG:32613',
    'crs_transform': [10, 0, 245470.38943568667, 0, -10, 4498062.075024601]},
   {'id': 'VH',
    'data_type': {'type': 'PixelType', 'precision': 'double'},
    'dimensions': [29008, 21488],
    'crs': 'EPSG:32613',
    'crs_transform': [10, 0, 245470.38943568667, 0, -10, 4498062.075024601]},
   {'id': 'angle',
    'data_type': {'type': 'PixelType', 'precision': 'float'},
    'dimensions': [21, 10],
    'crs': 'EPSG:32613',
    'crs_transform': [-12777.510236766713,
     -3931.9315816158196,
     534809.5794988564,
     2393.948875727132,
     -20007.636559526436,
     4449483.408209413]}],
  'version': 1698265761925472,
  'id': 'COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20200603T130946_20200603T131011_032853_03CE2C_9FA0',
  'properties': {'SNAP_Graph_Processing_Framework_GPF_vers': '7.0.3',
   'SLC_Processing_facility_org': 'ESA

In [373]:
images_meta_list = images.toList(count=20).getInfo()

In [374]:
def get_dates(image):
    return ee.Feature(None, {'aq_date': ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')})

In [375]:
dates = images.map(get_dates)

In [376]:
feat_list = dates.getInfo()['features']

In [377]:
aq_dates = [(d['id'], d['properties']['aq_date']) for d in feat_list]
aq_dates = sorted(aq_dates, key=lambda x: datetime.strptime(x[1], '%Y-%m-%d'))

In [378]:
aq_dates

[('S1A_IW_GRDH_1SDV_20200603T130946_20200603T131011_032853_03CE2C_9FA0',
  '2020-06-03'),
 ('S1A_IW_GRDH_1SDV_20200610T130139_20200610T130204_032955_03D135_658B',
  '2020-06-10'),
 ('S1A_IW_GRDH_1SDV_20200615T130947_20200615T131012_033028_03D365_A56F',
  '2020-06-15')]

## begin to detect change

In [379]:
import folium
from folium import plugins

In [380]:
# create a feature collection from i1, i2, i3
fc = ee.ImageCollection([i1, i2, i3])

# reproject to 32614
fc = fc.map(lambda img: img.reproject(crs='EPSG:32614', scale=10))

In [381]:
def specle_filterVV(img):
    vv = img.select('VV')
    vv_smoothed = vv.focal_median(50,'circle','meters').rename('VV_Filtered')
    return img.addBands(vv_smoothed)

def specle_filterVH(img):
    vh = img.select('VH')
    vh_smoothed = vh.focal_median(50,'circle','meters').rename('VH_Filtered')
    return img.addBands(vh_smoothed)

def dpsvi(img):
    dpsvi = ((img.select('VV_Filtered').add(img.select('VH_Filtered'))).divide(img.select('VV_Filtered'))).rename('DPSVI')
    return img.addBands(dpsvi)




In [382]:

def clip_img(self):
    """Clips a list of images."""
    return ee.Image(self).clip(aoi)


In [383]:

filterd_ic = fc.map(specle_filterVV)
filterd_ic = filterd_ic.map(specle_filterVH)

In [384]:
im_list = filterd_ic.toList(filterd_ic.size())
im_list_clipped = im_list.map(clip_img)

In [385]:
img1_vv = ee.Image(im_list_clipped.get(0)).select('VV_Filtered')
img1_vh = ee.Image(im_list_clipped.get(0)).select('VH_Filtered')
img2_vv = ee.Image(im_list_clipped.get(1)).select('VV_Filtered')
img2_vh = ee.Image(im_list_clipped.get(1)).select('VH_Filtered')

diff_vv = img1_vv.subtract(img2_vv).abs().rename('VV_diff')
diff_vh = img1_vh.subtract(img2_vh).abs().rename('VH_diff')

In [386]:
diff_vv_id_dict = diff_vv.getMapId()
diff_vh_id_dict = diff_vh.getMapId()



In [387]:
# create map object
map = folium.Map(location=[40.0150, -105.2705], zoom_start=12)

basemaps = {
    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps',
        overlay = True,
        control = True
    ),
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = True,
        control = True
    ),
    'Google Satellite Hybrid': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Esri Satellite': folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = True,
        control = True
    )
}

In [388]:
basemaps['Google Satellite Hybrid'].add_to(map)


diff_vv_folium = folium.raster_layers.TileLayer(
            tiles = diff_vv_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = 'diff_vv',
            overlay = True,
            control = True
            ).add_to(map)

diff_vh_folium = folium.raster_layers.TileLayer(
            tiles = diff_vh_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = 'diff_vh',
            overlay = True,
            control = True
            ).add_to(map)




In [389]:
map.add_child(folium.LayerControl())